In [13]:

# import .py files
import os
import pandas as pd
# import train_model
# import generate_synthetic
# import train_classifier
# import test_imgs
import split_dataset_v2 as split
import train_model as train

In [4]:
path_home = "/Users/hadare/Documents/CodingProjects/SyntheticEvaluation"
path_img =  path_home + "/images"
path_dataset = path_home + "/dataset"
path_exp = path_home + "/experiments"
path_models = path_home + "/models"
raw_data = path_home + "/data/mnist_images"

# Set the initial directory for experiments
initial_experiment_dir = "./experiments"
os.makedirs(initial_experiment_dir, exist_ok=True)  # Create directory if it doesn't exist

In [7]:
# set parameters for which train_size, gen_size, synthetic/real ratio, GAN train cutoff
# set initial dir for experiments

# Set parameters as lists of integer values
seed = 42
train_sizes = [1000, 2000, 3000]  # List of different training sizes
gen_sizes = [500, 1000, 1500]      # List of different generation sizes
synthetic_real_ratio = 0.5          # Ratio of synthetic to real data
gan_train_cutoff = 5000             # Number of GAN training iterations before switching
train_ratio = 0.8


# Print out the configured parameters for verification
print("Training Configuration:")
print(f"Train Size: {train_sizes}")
print(f"Generation Size: {gen_sizes}")
print(f"Synthetic/Real Ratio: {synthetic_real_ratio}")
print(f"StyleGAN2-ADA Training Cutoff: {gan_train_cutoff}")
# print(f"Experiment Run Directory: {experiment_run_dir}")
print("-" * 40)  # Separator for clarity


Training Configuration:
Train Size: [1000, 2000, 3000]
Generation Size: [500, 1000, 1500]
Synthetic/Real Ratio: 0.5
StyleGAN2-ADA Training Cutoff: 5000
----------------------------------------


In [4]:
# Use preprocessing.ipynb to create a proper dataset
# Distribute files to relevant subfolders + create JSON


In [14]:
# split into train/test

full_json = raw_data + '/dataset.json'

# split into test/train subsets
input_file = full_json



train_df, test_df = split.split_train_test(input_file, train_ratio, seed=seed)

train_output_file = f"{raw_data}/train_data.json"
test_output_file = f"{raw_data}/test_data.json"

split.save_data(train_df, train_output_file)
split.save_data(test_df, test_output_file)

split.print_class_distribution(train_df, "Train")
split.print_class_distribution(test_df, "Test")


Splitting data: 100%|██████████| 10/10 [00:00<00:00, 269.21class/s]

Train data statistics:
label
0    4738
1    5393
2    4766
3    4904
4    4673
5    4336
6    4734
7    5012
8    4680
9    4759
Name: count, dtype: int64
Test data statistics:
label
0    1185
1    1349
2    1192
3    1227
4    1169
5    1085
6    1184
7    1253
8    1171
9    1190
Name: count, dtype: int64


In [ ]:
# train MLP/CNN classifier, test for benchmark using test_imgs.py
# V

In [18]:
# for loop: create dir, create dataset, train model, generate syn-images, test

input_file = raw_data + '/train_data.json'

train_sizes = [1000]
for subset_size in train_sizes:
    # Create a name for the subset and the model
    model_name = f"model{subset_size // 1000}K"
    # subset the data according to size
    model_dir = path_models + '/' + model_name
    experiments_dir = os.path.join(model_dir, 'experiments')
    dataset_dir = os.path.join(model_dir, 'dataset')
    os.makedirs(dataset_dir, exist_ok=True)
    os.makedirs(experiments_dir, exist_ok=True)


    subset_df = split.subset_data(input_file, subset_size, seed=seed)
    output_file = f"{model_dir}/dataset_subset_size{subset_size}_seed{seed}.json"
    split.save_data(subset_df, output_file)
    split.print_class_distribution(subset_df, "Subset")
    
    
    print(f"Training {model_name} with {subset_size} images...")
    train.run_stylegan_training(subset_dir, experiments_dir, path_home, snap=10)


Subsetting classes: 100%|██████████| 10/10 [00:00<00:00, 1846.33class/s]

Subset data statistics:
label
0    100
1    100
2    100
3    100
4    100
5    100
6    100
7    100
8    100
9    100
Name: count, dtype: int64
Training model1K with 1000 images...


In [ ]:
# show results - relevant metric: avg/class accuracy, f1, precision, recall, AUC-ROC...
# graph/tabular

In [ ]:
# delete datasets (keep logs)